In [9]:
import cv2
import numpy as np
from mtcnn.mtcnn import MTCNN
import random   # Dummy classifier; replace with your model's code.
import time
import os
import platform
import subprocess
import requests  # For downloading images and videos
from bs4 import BeautifulSoup  # For parsing HTML content
import tkinter as tk
from urllib.parse import urljoin
import yt_dlp  # For extracting video URLs from arbitrary web pages
import warnings
warnings.filterwarnings('ignore')

# Hide the Tkinter root window (used only for potential dialogs)
root = tk.Tk()
root.withdraw()

# -------------------------------------------
# Configuration and Global Variables
# -------------------------------------------
DEEPFAKE_THRESHOLD = 0.5  # Confidence threshold to label a face as "Deepfake"

# Directory for saving snapshots and recordings.
capture_dir = os.path.join(os.getcwd(), "captures")
if not os.path.exists(capture_dir):
    os.makedirs(capture_dir)

# -------------------------------------------
# 1. Initialization and Dummy Classifier
# -------------------------------------------
detector = MTCNN()

def dummy_deepfake_classifier(face_img):
    """Simulate deepfake detection by returning a random confidence value."""
    return random.random()

def preprocess_face(face_img, target_size=(299, 299)):
    """Resize and normalize a face image."""
    try:
        face_img = cv2.resize(face_img, target_size)
    except Exception as e:
        print("Error resizing face:", e)
        return None
    face_img = face_img.astype("float32") / 255.0
    face_img = np.expand_dims(face_img, axis=0)  # Add batch dimension.
    return face_img

# -------------------------------------------
# 2. Functions for Online Image Processing
# -------------------------------------------
def retrieve_image_from_url(url):
    """
    Retrieve an image from any URL.
    If the URL directly returns an image, decode it.
    Otherwise, parse the HTML to find the first <img> tag and use that image.
    """
    try:
        response = requests.get(url)
        response.raise_for_status()
    except Exception as e:
        print("Error downloading URL:", e)
        return None

    content_type = response.headers.get("Content-Type", "").lower()
    if "image" in content_type:
        image_data = np.asarray(bytearray(response.content), dtype="uint8")
        image = cv2.imdecode(image_data, cv2.IMREAD_COLOR)
        return image
    elif "html" in content_type:
        soup = BeautifulSoup(response.text, "html.parser")
        img_tag = soup.find("img")
        if img_tag is None or not img_tag.get("src"):
            print("No image tag found in the HTML page.")
            return None
        img_url = img_tag["src"]
        if not img_url.startswith("http"):
            img_url = urljoin(response.url, img_url)
        try:
            response_img = requests.get(img_url)
            response_img.raise_for_status()
        except Exception as e:
            print("Error downloading image from parsed URL:", e)
            return None
        image_data = np.asarray(bytearray(response_img.content), dtype="uint8")
        image = cv2.imdecode(image_data, cv2.IMREAD_COLOR)
        return image
    else:
        print("Unsupported content type:", content_type)
        return None

def process_online_image():
    """
    Prompts for a URL, retrieves the image (directly or via HTML),
    performs deepfake detection, and displays the result.
    """
    url = input("Please enter the URL of an image (or webpage with an image): ").strip()
    if not url:
        print("No URL provided.")
        return

    image = retrieve_image_from_url(url)
    if image is None:
        print("Failed to retrieve a valid image from the provided URL.")
        return

    detections = detector.detect_faces(image)
    for detection in detections:
        x, y, w, h = detection['box']
        x, y = max(0, x), max(0, y)
        face = image[y:y+h, x:x+w]
        if face.size == 0:
            continue
        processed = preprocess_face(face)
        if processed is None:
            continue
        confidence = dummy_deepfake_classifier(processed)
        label = "Deepfake" if confidence >= DEEPFAKE_THRESHOLD else "Real"
        color = (0, 0, 255) if label == "Deepfake" else (0, 255, 0)
        cv2.rectangle(image, (x, y), (x+w, y+h), color, 2)
        cv2.putText(image, f"{label}: {confidence:.2f}", (x, y - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
    cv2.imshow("Online Image Deepfake Detection", image)
    print("Press any key in the image window to close it.")
    cv2.waitKey(0)
    cv2.destroyWindow("Online Image Deepfake Detection")

# -------------------------------------------
# 3. Functions for Online Video Processing using yt_dlp
# -------------------------------------------
def extract_direct_video_url(url):
    """
    Uses yt_dlp to extract a direct video URL from any web source.
    This works for many popular sites (YouTube, Vimeo, etc.) as well as generic pages.
    """
    ydl_opts = {
        'quiet': True,
        'skip_download': True,
        'forceurl': True,  # Ask for a URL only.
    }
    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info_dict = ydl.extract_info(url, download=False)
            # If there are multiple formats, choose one (e.g., the first available)
            if 'formats' in info_dict:
                # You can further refine the selection here (e.g., highest resolution)
                return info_dict['formats'][0]['url']
            else:
                return info_dict.get('url', None)
    except Exception as e:
        print("Error extracting video URL with yt_dlp:", e)
        return None

def process_online_video():
    """
    Prompts for a video URL, uses yt_dlp to extract a direct video URL,
    downloads the video, and processes it frame by frame for deepfake detection.
    """
    url = input("Please enter the URL of a video: ").strip()
    if not url:
        print("No URL provided.")
        return

    direct_video_url = extract_direct_video_url(url)
    if direct_video_url is None:
        print("Failed to retrieve a direct video URL from the provided URL.")
        return

    # Download the video file to a temporary path.
    temp_video_path = os.path.join(capture_dir, "temp_video.mp4")
    try:
        with requests.get(direct_video_url, stream=True) as r:
            r.raise_for_status()
            with open(temp_video_path, 'wb') as f:
                for chunk in r.iter_content(chunk_size=8192):
                    if chunk:
                        f.write(chunk)
    except Exception as e:
        print("Error downloading video:", e)
        return

    # Open and process the downloaded video.
    video_cap = cv2.VideoCapture(temp_video_path)
    if not video_cap.isOpened():
        print("Error: Could not open the downloaded video file.")
        os.remove(temp_video_path)
        return

    print("Processing online video...")
    while True:
        ret, frame = video_cap.read()
        if not ret:
            break
        detections = detector.detect_faces(frame)
        for detection in detections:
            x, y, w, h = detection['box']
            x, y = max(0, x), max(0, y)
            face = frame[y:y+h, x:x+w]
            if face.size == 0:
                continue
            processed = preprocess_face(face)
            if processed is None:
                continue
            confidence = dummy_deepfake_classifier(processed)
            label = "Deepfake" if confidence >= DEEPFAKE_THRESHOLD else "Real"
            color = (0, 0, 255) if label == "Deepfake" else (0, 255, 0)
            cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
            cv2.putText(frame, f"{label}: {confidence:.2f}", (x, y - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
        cv2.imshow("Online Video Deepfake Detection", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    video_cap.release()
    cv2.destroyWindow("Online Video Deepfake Detection")
    os.remove(temp_video_path)
    print("Finished processing online video.")

# -------------------------------------------
# 4. Capture Live Webcam Feed and Deepfake Detection
# -------------------------------------------
def capture_live_video():
    """Capture video feed from webcam, detect deepfakes, and save snapshots."""
    webcam = cv2.VideoCapture(0)
    if not webcam.isOpened():
        print("Error: Could not access webcam.")
        return
    
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for MP4 format
    out = cv2.VideoWriter(os.path.join(capture_dir, 'captured_video.mp4'), fourcc, 20.0, (640, 480))

    while True:
        ret, frame = webcam.read()
        if not ret:
            break

        detections = detector.detect_faces(frame)
        for detection in detections:
            x, y, w, h = detection['box']
            x, y = max(0, x), max(0, y)
            face = frame[y:y+h, x:x+w]
            if face.size == 0:
                continue
            processed = preprocess_face(face)
            if processed is None:
                continue
            confidence = dummy_deepfake_classifier(processed)
            label = "Deepfake" if confidence >= DEEPFAKE_THRESHOLD else "Real"
            color = (0, 0, 255) if label == "Deepfake" else (0, 255, 0)
            cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
            cv2.putText(frame, f"{label}: {confidence:.2f}", (x, y - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

        # Show the frame with detection info
        cv2.imshow('Webcam Deepfake Detection', frame)
        out.write(frame)  # Save video frame to the output file
        
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):  # Press 'q' to quit
            break
        if key == ord('c'):  # Press 'c' to capture a snapshot
            snapshot_filename = os.path.join(capture_dir, f"snapshot_{int(time.time())}.png")
            cv2.imwrite(snapshot_filename, frame)
            print(f"Snapshot saved to: {snapshot_filename}")

    webcam.release()
    out.release()
    cv2.destroyAllWindows()
    print("Video capture finished.")

# -------------------------------------------
# Main Program
# -------------------------------------------
def main():
    print("Deepfake Detection System")
    print("Choose an option:")
    print("1. Process an online image")
    print("2. Process an online video")
    print("3. Capture live webcam video")

    choice = input("Enter your choice (1-3): ").strip()
    if choice == '1':
        process_online_image()
    elif choice == '2':
        process_online_video()
    elif choice == '3':
        capture_live_video()
    else:
        print("Invalid choice. Exiting...")

if __name__ == "__main__":
    main()

Deepfake Detection System
Choose an option:
1. Process an online image
2. Process an online video
3. Capture live webcam video
Enter your choice (1-3): 1
Please enter the URL of an image (or webpage with an image): https://in.images.search.yahoo.com/images/view;_ylt=AwrKGFUmlKRndy8hK3O9HAx.;_ylu=c2VjA3NyBHNsawNpbWcEb2lkAzJiYmUzZDI2MWVlY2U1ZDAwZmJjZGFkMmQ4NWM2MmUyBGdwb3MDMwRpdANiaW5n?back=https%3A%2F%2Fin.images.search.yahoo.com%2Fsearch%2Fimages%3Fp%3Dsachin%2Btendulkar%26type%3DE210IN826G91833%26fr%3Dmcafee%26h%3D800%26tt%3DSachin%2BTendulkar%25E2%2580%2599s%2Bphilanthropic%2Bgesture%253B%2Bdonates%2Bmedical%2Bequipment%2Bto%2B...%26w%3D1200%26imgurl%3Dhttps%253A%252F%252Fnenow.in%252Fwp-content%252Fuploads%252F2020%252F11%252FSachin_Tendulkar_AFP.jpg%26rurl%3Dhttps%253A%252F%252Fnenow.in%252Fnorth-east-news%252Fassam%252Fsachin-tendulkars-philanthropic-gesture-donates-medical-equipment-to-assam-hospital.html%26turl%3Dhttps%253A%252F%252Ftse1.mm.bing.net%252Fth%253Fid%253DOIP.6WAfu25k